# Animated primary energy consumption versus time

The data are from the [statistical review of world energy](https://www.energyinst.org/statistical-review). For the animation, we will use the [ipyvizzu](https://ipyvizzu.vizzuhq.com/latest/) library.

In [ ]:
# run this cell if you haven't installed ipyvizzu
import sys
!{sys.executable} -m pip install ipyvizzu

In [ ]:
# import pandas and ipyvizzu libraries
import pandas as pd
import ipyvizzu as vz

In [ ]:
# Read primary energy consumption per capita

# file path
path = "../data/Statistical Review of World Energy Data.xlsx"
# read data from third row and drop last 13 rows
pec_cap = pd.read_excel(path, sheet_name = "PE cons per capita GJ", header=2, skipfooter=13) 
# remove empty rows
pec_cap.dropna(inplace=True)
# name of first column
name_fc = pec_cap.columns[0]
# remove rows containing "Total"
pec_cap.drop(pec_cap[pec_cap[name_fc].str.contains("Total")].index, inplace=True)
# remove last two columns
pec_cap.drop(columns=pec_cap.columns[-2:], axis=1,  inplace=True)
# make first column the index of the DataFrame
pec_cap.set_index(name_fc, inplace=True)
# make all columns numeric
pec_cap = pec_cap.apply(pd.to_numeric, errors="coerce")
# sort DataFrame by 2022 column in a descending order
pec_cap.sort_values(by=[2022], ascending=False, inplace=True)
# make a list of the top 25 countries
countries = pec_cap.index.values.tolist()[0:25]

pec_cap.head()

In [ ]:
# I want these data instead as three columns: Country, Year, and Primary energy consumption

# make series with stacked year columns
pec_cap_series = pec_cap.stack()
# make DataFrame with series
pec_cap_stack = pd.DataFrame(pec_cap_series)
# rename first column 
pec_cap_stack.rename(columns={0: name_fc}, inplace=True)
# add Country column from DataFrame index
pec_cap_stack["Country"] = pec_cap_stack.index.get_level_values(0)
# add a Year column from DataFrame index
pec_cap_stack["Year"] = pec_cap_stack.index.get_level_values(1)
# reset index
pec_cap_stack = pec_cap_stack.reset_index(drop=True)

pec_cap_stack.head()

In [ ]:
# Filter to extract the top countries
pec_cap_stack_filter = (pec_cap_stack["Country"].isin(countries))

# add DataFrame of selected countries to the ipyvizzu model
data = vz.Data()
data.add_df(pec_cap_stack[pec_cap_stack_filter])

In [ ]:
# Choosing the x and y, x label, and property controling color
config = {
    "channels": {
        "y": {
            "set": ["Country"],
        },
        "x": {"set": [name_fc]},
        "label": {"set": [name_fc]},
        "color": {"set": ["Country"]},
    },
    "sort": "byValue",
}

In [ ]:
# labels and padding

# ipyvizzu style
style = vz.Style(
    {
        "plot": {
            "paddingLeft": 150,
            "paddingTop": 25,
            "yAxis": {
                "color": "#ffffff00",
                "label": {"paddingRight": 10},
            },
            "xAxis": {
                "title": {"color": "#ffffff00"},
                "label": {
                    "color": "#ffffff00",
                    "numberFormat": "grouped",
                },
            },
        },
    }
)

In [ ]:
# create the chart object
chart = vz.Chart(display=vz.DisplayTarget.END)
#chart = vz.Chart(width="840px", height="640px")

# include data and style previously defined
chart.animate(data, style)

# loop over the years
for year in range(1965, 2024):
    # chart title
    config["title"] = f"Primary energy consumption per capita (Gigajoules) in {year}"
    # chart
    chart.animate(
        # data for year
        vz.Data.filter(f"parseInt(record.Year) == {year}"),
        # apply configuration
        vz.Config(config),
        # duration, etc.
        duration=1,
        x={"easing": "linear", "delay": 0},
        y={"delay": 0},
        show={"delay": 0},
        hide={"delay": 0},
        title={"duration": 0, "delay": 0},
    )